# Deltares Software Days 2022: Hydrolib-core demo

Hydrolib-core is a pip-installable python package. It consists of wrappers around the D-HYDRO model files.
Classes in Hydrolib-core are hierarchically organized by file type, and can be accessed via `hydrolib.core.io.<modeltype>.<filetype>.models`.
File objects in Hydrolib-core reflect the same structure as the file contents. 

In this demo, we will load a Flow FM model, make some changes to the model and save it again. 

## Step 0) Importing the modules 

In [31]:
from hydrolib.core.io.mdu.models import FMModel
from hydrolib.core.io.structure.models import Weir, FlowDirection
from pathlib import Path

## Step 1) Loading a Flow FM model

In [32]:
demo_data_folder = Path.cwd() / "data"
mdu_file_path = demo_data_folder /"FlowFM.mdu"

model = FMModel(filepath=mdu_file_path)

## Step 2) Inspecting the model

In [33]:
# Show the hierachy tree of the model
model.show_tree()

# Get the structure file of the model. 
# Note that you can have multiple structure files, but this model has only one
assert model.geometry.structurefile is not None
structure_file = model.geometry.structurefile[0]

# Show the list of all the structures
print(f"Number of structures: {len(structure_file.structure)}")
print(structure_file.structure)

  d:\checkouts\HydroLib-core\HydroLib-core_1\docs\tutorials\dsd_2022_demo\data\FlowFM.mdu
    Geometry
     ∟ FlowFM_net.nc
     ∟ structures.ini
     ∟ initialFields.ini
     ∟ roughness-Channels.ini
     ∟ roughness-Main.ini
     ∟ roughness-Sewer.ini
     ∟ roughness-FloodPlain1.ini
     ∟ roughness-FloodPlain2.ini
     ∟ crsdef.ini
     ∟ crsloc.ini
     ∟ nodeFile.ini
    ExternalForcing
     ∟ FlowFM_bnd.ext
       ∟ Lateral
         ∟ FlowFM_lateral_sources.bc
       ∟ Lateral
         ∟ FlowFM_lateral_sources.bc
       ∟ Lateral
       ∟ Lateral
       ∟ Lateral
         ∟ FlowFM_lateral_sources.bc
       ∟ Lateral
         ∟ FlowFM_lateral_sources.bc
       ∟ Lateral
         ∟ FlowFM_lateral_sources.bc
    Output
     ∟ obsFile1D_obs.ini
Number of structures: 10
[Pump(comments=Comments(id=None, name=None, polylinefile='*.pli; Polyline geometry definition for 2D structure.', branchid=None, chainage=None, numcoordinates='Number of values in xCoordinates and yCoordinates', xcoor

## Step 3) Adjusting the model

### Adding a new structure

In [34]:
# Create a new weir.
weir = Weir(
    id="DV_1471", 
    branchId="458", 
    chainage=106.277467, 
    allowedFlowDir=FlowDirection.both, 
    crestLevel=0.400, 
    crestWidth=40.000, 
    corrCoeff=1.000, 
    useVelocityHeight=True
)

# It is also possible to create objects, by passing a dictionary.
weir_data = {
    "id": "DV_1471", 
    "branchId": "458", 
    "chainage": 106.277467, 
    "allowedFlowDir": FlowDirection.both, 
    "crestLevel": 0.400, 
    "crestWidth": 40.000, 
    "corrCoeff": 1.000, 
    "useVelocityHeight": True
}
weir = Weir(**weir_data)

# Add the weir to the model
structure_file.structure.append(weir)

print(f"Number of structures: {len(structure_file.structure)}")


Number of structures: 11


### Adjust some parameters of the model

In [36]:
model.physics.backgroundsalinity = 30 # [ppt]
model.physics.backgroundwatertemperature = 6 # [°C]
model.time.dtuser = 900 # [s]